In [1]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability trick
    return e_x / e_x.sum(axis=1, keepdims=True)

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Global storage (simulating Redis)
storage = {}

class Neuron:
    def __init__(self, weights, bias, activation, layer_id, neuron_id, is_final_layer=False):
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        # Ensure inputs match expected shape
        input_values = np.array([storage[inp] for inp in inputs if inp in storage])  # shape (batch_size, num_inputs)
        input_values = np.stack(input_values, axis=1)  # Reshape properly

        if input_values.shape[1] != self.weights.shape[0]:
            raise ValueError(f"Neuron {self.layer_id}_{self.neuron_id} received {input_values.shape[1]} inputs "
                             f"but expected {self.weights.shape[0]}")

        # Compute activation
        z = np.dot(input_values, self.weights) + self.bias  # shape (batch_size,)
        output = z if self.is_final_layer else self.activation_func(z)

        # Store output
        storage[f"{self.layer_id}_{self.neuron_id}"] = output
        return output

class Layer:
    def __init__(self, layer_id, neurons):
        self.layer_id = layer_id
        self.neurons = neurons

    def forward(self, input_keys):
        inputs = [key for key in input_keys if key in storage]  # Ensure valid keys
        outputs = [neuron.forward(inputs) for neuron in self.neurons]

        return outputs

# Build network dynamically from JSON
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))  # Ensure correct order

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        layer_neurons = []

        for j, node in enumerate(layer_info['nodes']):
            neuron = Neuron(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                layer_id=layer_name,
                neuron_id=j,
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            layer_neurons.append(neuron)

        layers.append(Layer(layer_name, layer_neurons))

    return layers

# Forward pass for batch processing
def forward_pass(layers, input_data):
    batch_size = input_data.shape[0]
    storage.clear()  # Reset storage

    # Initialize input storage for the batch
    for i in range(input_data.shape[1]):  # Iterate over feature dimensions
        storage[f"input_{i}"] = input_data[:, i]  # Store batch of input features

    # Process each layer
    input_keys = [f"input_{i}" for i in range(input_data.shape[1])]

    for layer in layers:
        layer.forward(input_keys)
        input_keys = [f"{layer.layer_id}_{j}" for j in range(len(layer.neurons))]

    # Retrieve final layer outputs
    final_outputs = np.array([storage[f"{layers[-1].layer_id}_{j}"] for j in range(len(layers[-1].neurons))]).T  # Transpose to (batch_size, num_classes)

    # Apply softmax at the final layer
    final_outputs = softmax(final_outputs)

    return np.argmax(final_outputs, axis=1)  # Return predicted classes for batch

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load MNIST dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Process images in batches
batch_size = 10
images, labels = [], []

for i in range(batch_size):  # Collect batch
    image, label = mnist_test[i]
    images.append(image.view(-1).numpy())  # Flatten 28x28 image to 784 input vector
    labels.append(label)

images = np.stack(images)  # Convert list to array of shape (batch_size, 784)

# Run batch prediction
predicted_classes = forward_pass(network, images)

# Print results
for i, (true_label, pred_label) in enumerate(zip(labels, predicted_classes)):
    print(f"Image {i}: Label = {true_label}, Predicted = {pred_label}")


Image 0: Label = 7, Predicted = 7
Image 1: Label = 2, Predicted = 2
Image 2: Label = 1, Predicted = 1
Image 3: Label = 0, Predicted = 0
Image 4: Label = 4, Predicted = 4
Image 5: Label = 1, Predicted = 1
Image 6: Label = 4, Predicted = 4
Image 7: Label = 9, Predicted = 9
Image 8: Label = 5, Predicted = 6
Image 9: Label = 9, Predicted = 9


#### All at once

In [3]:
import json
import numpy as np
from torchvision import datasets, transforms

# Load JSON file
def load_network(filename):
    with open(filename, 'r') as f:
        return json.load(f)

# Activation functions
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability trick
    return e_x / e_x.sum(axis=1, keepdims=True)

ACTIVATIONS = {
    "relu": relu,
    "softmax": softmax
}

# Global storage (simulating Redis)
storage = {}

class Neuron:
    def __init__(self, weights, bias, activation, layer_id, neuron_id, is_final_layer=False):
        self.weights = np.array(weights)
        self.bias = np.array(bias)
        self.activation_func = None if is_final_layer else ACTIVATIONS.get(activation, relu)
        self.layer_id = layer_id
        self.neuron_id = neuron_id
        self.is_final_layer = is_final_layer

    def forward(self, inputs):
        # Ensure inputs match expected shape
        input_values = np.array([storage[inp] for inp in inputs if inp in storage])  # shape (batch_size, num_inputs)
        input_values = np.stack(input_values, axis=1)  # Reshape properly

        if input_values.shape[1] != self.weights.shape[0]:
            raise ValueError(f"Neuron {self.layer_id}_{self.neuron_id} received {input_values.shape[1]} inputs "
                             f"but expected {self.weights.shape[0]}")

        # Compute activation
        z = np.dot(input_values, self.weights) + self.bias  # shape (batch_size,)
        output = z if self.is_final_layer else self.activation_func(z)

        # Store output
        storage[f"{self.layer_id}_{self.neuron_id}"] = output
        return output

class Layer:
    def __init__(self, layer_id, neurons):
        self.layer_id = layer_id
        self.neurons = neurons

    def forward(self, input_keys):
        inputs = [key for key in input_keys if key in storage]  # Ensure valid keys
        outputs = [neuron.forward(inputs) for neuron in self.neurons]

        return outputs

# Build network dynamically from JSON
def build_network(json_data):
    layers = []
    sorted_layers = sorted(json_data.keys(), key=lambda x: int(x.split('_')[-1]))  # Ensure correct order

    for i, layer_name in enumerate(sorted_layers):
        layer_info = json_data[layer_name]
        layer_neurons = []

        for j, node in enumerate(layer_info['nodes']):
            neuron = Neuron(
                weights=np.array(node['weights']),
                bias=np.array(node['biases']),
                activation=node['activation'],
                layer_id=layer_name,
                neuron_id=j,
                is_final_layer=(i == len(sorted_layers) - 1)
            )
            layer_neurons.append(neuron)

        layers.append(Layer(layer_name, layer_neurons))

    return layers

# Forward pass for full dataset
def forward_pass(layers, input_data):
    batch_size = input_data.shape[0]
    storage.clear()  # Reset storage

    # Initialize input storage for the entire dataset
    for i in range(input_data.shape[1]):  # Iterate over feature dimensions
        storage[f"input_{i}"] = input_data[:, i]  # Store batch of input features

    # Process each layer
    input_keys = [f"input_{i}" for i in range(input_data.shape[1])]

    for layer in layers:
        layer.forward(input_keys)
        input_keys = [f"{layer.layer_id}_{j}" for j in range(len(layer.neurons))]

    # Retrieve final layer outputs
    final_outputs = np.array([storage[f"{layers[-1].layer_id}_{j}"] for j in range(len(layers[-1].neurons))]).T  # Transpose to (batch_size, num_classes)

    # Apply softmax at the final layer
    final_outputs = softmax(final_outputs)

    return np.argmax(final_outputs, axis=1)  # Return predicted classes for full dataset

# Load network
data = load_network("node_based_model.json")
network = build_network(data)

# Load entire MNIST test dataset with correct normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize to [-1, 1]
])
mnist_test = datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# Convert full dataset to NumPy arrays
images = np.stack([mnist_test[i][0].view(-1).numpy() for i in range(len(mnist_test))])  # (num_samples, 784)
labels = np.array([mnist_test[i][1] for i in range(len(mnist_test))])  # (num_samples,)

# Run prediction for entire dataset
predicted_classes = forward_pass(network, images)

# Evaluate Accuracy
accuracy = np.mean(predicted_classes == labels)
print(f"Accuracy on full MNIST test set: {accuracy * 100:.2f}%")

# Print some sample predictions
for i in range(10):
    print(f"Image {i}: True Label = {labels[i]}, Predicted = {predicted_classes[i]}")


Accuracy on full MNIST test set: 96.73%
Image 0: True Label = 7, Predicted = 7
Image 1: True Label = 2, Predicted = 2
Image 2: True Label = 1, Predicted = 1
Image 3: True Label = 0, Predicted = 0
Image 4: True Label = 4, Predicted = 4
Image 5: True Label = 1, Predicted = 1
Image 6: True Label = 4, Predicted = 4
Image 7: True Label = 9, Predicted = 9
Image 8: True Label = 5, Predicted = 6
Image 9: True Label = 9, Predicted = 9
